# Configurações iniciais

In [130]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from crewai_tools import TXTSearchTool
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from datetime import datetime
import re
import pandas as pd
import psycopg2

# Definir os parâmetros de conexão com o PostgreSQL
usuario = 'postgres'
senha = '87a9f05e0b51d376b83d582111c280da'
host = '147.79.110.26'  # Endereço do servidor do banco
porta = '5432'           # Porta padrão do PostgreSQL
banco_de_dados = 'botai_dev'
# client = OpenAI(api_key="sk-dGwU-tTvhnrFog5R78lzLlP5FjaOOlWsz_Qbb4CdgoT3BlbkFJ_iRd_S5W91m7pOY9i8oT_E8wHZwgK2EiqMROjF0iIA")

load_dotenv()

OPEN_AI_KEY = os.getenv("OPEN_AI_KEY")

csv_folder = r"Treinamento/ChatHistory/byData/024509451509359969@g.us"

## Métodos para carregar tabelas

In [2]:
# Função para criar um DataFrame a partir de uma tabela do banco de dados
def carregar_tabela_postgresql(nome_tabela):
    try:
        # Estabelecendo conexão com o banco de dados
        conn = psycopg2.connect(
            dbname=banco_de_dados,
            user=usuario,
            password=senha,
            host=host,
            port=porta
        )
        
        # Query para selecionar todos os dados da tabela desejada
        query = f"SELECT * FROM {nome_tabela};"
        
        # Lendo os dados diretamente no DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Fechando a conexão
        conn.close()
        
        return df
    
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

## Função para criar um DataFrame a partir de uma tabela do banco de dados, permitindo parametrizar a condição WHERE
def carregar_tabela_com_where(nome_tabela, where_condition):
    try:
        # Estabelecendo conexão com o banco de dados
        conn = psycopg2.connect(
            dbname=banco_de_dados,
            user=usuario,
            password=senha,
            host=host,
            port=porta
        )
        
        # Query para selecionar os dados da tabela desejada com a condição WHERE
        query = f"SELECT * FROM {nome_tabela} WHERE {where_condition};"
        
        # Lendo os dados diretamente no DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Fechando a conexão
        conn.close()
        
        return df
    
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None


## Carregando dados

In [3]:
chat_history = carregar_tabela_com_where("chat_history_t","created_at::date = (CURRENT_DATE - INTERVAL '3 day')") #Coreto = 1 day
people = carregar_tabela_postgresql("people")
groups = carregar_tabela_postgresql("groups")
packages = carregar_tabela_postgresql("packages")
departments = carregar_tabela_postgresql("departments")

chat_history.shape

/tmp/ipykernel_1851117/3732391048.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_1851117/3732391048.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


(62, 23)

## Unindo gupos e pacotes

In [4]:
groups = groups.rename(columns={'description': 'group_description', 'created_at': 'group_created'})
cols = ['group_id','group_jid', 'group_name', 'package_id', 'project_id', 'group_description', 'group_created']
groups = groups[cols]

groups.sample()

,group_id,group_jid,group_name,package_id,project_id,group_description,group_created
2,4,171622593129764632@g.us,Plinio-Vibefortech,2,2,None,2024-10-04 02:25:11.187973


In [5]:
packages = packages.rename(columns={'description': 'package_description'})
cols = ['package_id', 'package_name']
packages = packages[cols]
packages.sample()

,package_id,package_name
0,1,Pacote Especial


In [6]:
groups.shape

(12, 7)

In [7]:
grupos_pacotes = pd.merge(groups, packages, on='package_id', how='left')
grupos_pacotes

,group_id,group_jid,group_name,package_id,project_id,group_description,group_created,package_name
0,2,024509451509359969@g.us,Maiara-Mayavant,2,2,None,2024-10-04 02:25:11.187973,Pacote Global Dois Em Um
1,3,025587043275878954@g.us,EduardoeNicolas-MulherdaWeb,1,2,None,2024-10-04 02:25:11.187973,Pacote Especial
2,4,171622593129764632@g.us,Plinio-Vibefortech,2,2,None,2024-10-04 02:25:11.187973,Pacote Global Dois Em Um
3,5,176569235456585413@g.us,SamueleLeonardo-Linqtec,1,2,None,2024-10-04 02:25:11.187973,Pacote Especial
4,6,220191283813053475@g.us,Dr.MuriloeMurilo-Dynathlete,2,2,None,2024-10-04 02:25:11.187973,Pacote Global Dois Em Um
5,7,319892153194284268@g.us,Pedro-StoreForKids,1,2,None,2024-10-04 02:25:11.187973,Pacote Especial
6,8,370613401684902859@g.us,MarianaeRita-ExpressodoAconchego,1,2,None,2024-10-04 02:25:11.187973,Pacote Especial
7,9,436283272096070764@g.us,TâniaeThaissa-TajuvanHouse,1,2,None,2024-10-04 02:25:11.187973,Pacote Especial
8,10,765321137722616028@g.us,Nitiel-Apertto,2,2,None,2024-10-04 02:25:11.187973,Pacote Global Dois Em Um
9,11,960751260776592837@g.us,Tiago-AkkiShop,2,2,None,2024-10-04 02:25:11.187973,Pacote Global Dois Em Um


## Unindo pessoas, departamentos e histórico

In [8]:
people.sample()

,person_id,remote_jid,person_name,nickname,status,department_id,is_client,description,created_at,updated_at
15,2009,5511960845692,Cliente 11960845692,None,ativo,NaN,True,None,2024-10-04 02:54:55.428971,2024-10-04 02:54:55.428971


In [9]:
# people = people.rename(columns={'description': 'package_description'})
cols = ['remote_jid', 'person_name','nickname','department_id','is_client']
people = people[cols]
people.sample()

,remote_jid,person_name,nickname,department_id,is_client
30,5567962177849,Veronica Goodds Teste,Veronica,3.0,False


In [10]:
departments.sample()

,department_id,department_name,status,description,created_at,updated_at
1,2,Marketing,None,Departamento responsável pelas estratégias de ...,2024-10-03 12:57:49.363868,2024-10-03 12:57:49.363868


In [11]:
departments = departments.rename(columns={'description': 'department_description'})
cols = ['department_id', 'department_name','department_description']
departments = departments[cols]
departments.sample()

,department_id,department_name,department_description
0,1,Desenvolvimento,Departamento responsável pelo desenvolvimento ...


In [12]:
print(people.shape)
print(departments.shape)

(32, 5)
(4, 3)


In [13]:
person_dep = pd.merge(people, departments, on='department_id', how='left')
person_dep.sample()

,remote_jid,person_name,nickname,department_id,is_client,department_name,department_description
4,553183135034,Cliente 3183135034,None,NaN,True,NaN,NaN


In [14]:
print(person_dep.shape)

(32, 7)


In [15]:
cols = ['remote_jid', 'person_name','nickname','is_client','department_name']
person_dep = person_dep[cols]
person_dep.sample()

,remote_jid,person_name,nickname,is_client,department_name
28,5587936169444,Stephano Goodds Teste,Stephano,False,Desenvolvimento


In [16]:
person_dep[person_dep['remote_jid']=='5518919879626']

,remote_jid,person_name,nickname,is_client,department_name
22,5518919879626,Giovana Goodds Teste,Giovana,False,Relacionamento


In [17]:
chat_history.sample()

,message_id,event,instance,receiver,groupJid,sender_id,from_me,push_name,message_type,message_content,...,latitude,longitude,sequence_number,display_name,vcard,poll_data,source,reaction_message_id,quoted_message,created_at
23,3761ef81-1fce-4d96-9add-a88e39418940,messages.upsert,BotAI,5522992848074,025587043275878954@g.us,554191930514,None,Cliente 4191930514,texto,Procura*,...,None,None,None,None,None,None,None,None,None,2024-10-03 17:02:00


## Unindo chat_history a grupos e pessoas

In [18]:
chat_history.shape

(62, 23)

In [19]:
person_dep.shape

(32, 5)

In [20]:
# Exemplo: realizar o merge entre chat_history e person_dep usando as colunas específicas
chat_history = pd.merge(
    chat_history,
    person_dep,
    left_on='sender_id',  # Coluna do DataFrame chat_history
    right_on='remote_jid',  # Coluna do DataFrame person_dep
    how='left'  # Tipo de merge
)

chat_history.shape

(62, 28)

In [21]:
chat_history.sample()

,message_id,event,instance,receiver,groupJid,sender_id,from_me,push_name,message_type,message_content,...,poll_data,source,reaction_message_id,quoted_message,created_at,remote_jid,person_name,nickname,is_client,department_name
10,445fc0ec-0ee5-4ae5-ab41-7ad3a38663e0,messages.upsert,BotAI,5522992848074,025587043275878954@g.us,5558980748460,None,Luiza Goodds - Central de atendimento,áudio,‎PTT-20240130-WA0090.mp3,...,None,None,None,None,2024-10-03 16:48:00,5558980748460,Luiza Goodds - Central de atendimento Teste,Luiza,False,Relacionamento


In [22]:
# Exemplo: realizar o merge entre chat_history e person_dep usando as colunas específicas
chat_groups_history = pd.merge(
    chat_history,
    grupos_pacotes,
    left_on='groupJid',  # Coluna do DataFrame chat_history
    right_on='group_jid',  # Coluna do DataFrame person_dep
    how='left'  # Tipo de merge
)

chat_groups_history.shape

(62, 36)

In [23]:
chat_groups_history.sample()

,message_id,event,instance,receiver,groupJid,sender_id,from_me,push_name,message_type,message_content,...,is_client,department_name,group_id,group_jid,group_name,package_id,project_id,group_description,group_created,package_name
21,2cd22555-467c-421b-bcc9-a4e63bd948ec,messages.upsert,BotAI,5522992848074,025587043275878954@g.us,554191930514,None,Cliente 4191930514,texto,Estudamos o mercado,...,True,NaN,3,025587043275878954@g.us,EduardoeNicolas-MulherdaWeb,1,2,None,2024-10-04 02:25:11.187973,Pacote Especial


In [24]:
chat_groups_history.columns

Index(['message_id', 'event', 'instance', 'receiver', 'groupJid', 'sender_id',
       'from_me', 'push_name', 'message_type', 'message_content', 'media_url',
       'base64_data', 'reaction', 'latitude', 'longitude', 'sequence_number',
       'display_name', 'vcard', 'poll_data', 'source', 'reaction_message_id',
       'quoted_message', 'created_at', 'remote_jid', 'person_name', 'nickname',
       'is_client', 'department_name', 'group_id', 'group_jid', 'group_name',
       'package_id', 'project_id', 'group_description', 'group_created',
       'package_name'],
      dtype='object')

In [25]:
# Definir as colunas que você deseja remover
colunas_a_remover = ['instance', 'receiver', 'sender_id','push_name','base64_data','sequence_number','group_jid','package_id','group_description','from_me']

# Remover as colunas do DataFrame
chat_groups_history = chat_groups_history.drop(columns=colunas_a_remover)

# Exibir um sample do DataFrame atualizado
chat_groups_history.sample()

,message_id,event,groupJid,message_type,message_content,media_url,reaction,latitude,longitude,display_name,...,remote_jid,person_name,nickname,is_client,department_name,group_id,group_name,project_id,group_created,package_name
8,29fce13a-89f7-4774-a69c-82611c68dd65,messages.upsert,025587043275878954@g.us,texto,"Prazer Giovanna, sou do administrativo da Goodds.",None,None,None,None,None,...,5518919879626,Giovana Goodds Teste,Giovana,False,Relacionamento,3,EduardoeNicolas-MulherdaWeb,2,2024-10-04 02:25:11.187973,Pacote Especial


## Separando os grupos

In [26]:
chat_groups_history.columns

Index(['message_id', 'event', 'groupJid', 'message_type', 'message_content',
       'media_url', 'reaction', 'latitude', 'longitude', 'display_name',
       'vcard', 'poll_data', 'source', 'reaction_message_id', 'quoted_message',
       'created_at', 'remote_jid', 'person_name', 'nickname', 'is_client',
       'department_name', 'group_id', 'group_name', 'project_id',
       'group_created', 'package_name'],
      dtype='object')

In [27]:
dfs_groupJid = {}

#  Obter todos os valores únicos na coluna groupJid
grupos = chat_groups_history['groupJid'].unique()

# Para cada valor único, filtrar o DataFrame e armazenar em um novo DataFrame
for g in grupos:
    df_grupos = chat_groups_history[chat_groups_history['groupJid'] == g]
    dfs_groupJid[g] = df_grupos
    print(g)

dfs_groupJid['025587043275878954@g.us'].shape

024509451509359969@g.us
025587043275878954@g.us


(55, 26)

In [28]:
dfs_groupJid['025587043275878954@g.us'].columns

Index(['message_id', 'event', 'groupJid', 'message_type', 'message_content',
       'media_url', 'reaction', 'latitude', 'longitude', 'display_name',
       'vcard', 'poll_data', 'source', 'reaction_message_id', 'quoted_message',
       'created_at', 'remote_jid', 'person_name', 'nickname', 'is_client',
       'department_name', 'group_id', 'group_name', 'project_id',
       'group_created', 'package_name'],
      dtype='object')

In [29]:
dfs_groupJid['025587043275878954@g.us'].shape

(55, 26)

In [86]:
# Define o caminho do diretório que você deseja criar
directory_path = 'conversas'

# Cria o diretório se ele não existir
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Diretório '{directory_path}' criado.")
else:
    print(f"O diretório '{directory_path}' já existe.")

# Supondo que dfs_groupJid seja o seu DataFrame
file_name = 'output.txt'  # Nome do arquivo
file_path = os.path.join(directory_path, file_name)  # Caminho completo para o arquivo

# Transformando o DataFrame em um arquivo TXT
dfs_groupJid['025587043275878954@g.us'].to_csv(file_path, sep='\t', index=False)

print(f"DataFrame salvo em {file_path}")


O diretório 'conversas' já existe.
DataFrame salvo em conversas/output.txt


In [87]:
# Obter valores 
group_id = dfs_groupJid['025587043275878954@g.us']['group_id'].unique().tolist()
group_id = group_id[0]

group_jid = dfs_groupJid['025587043275878954@g.us']['groupJid'].unique().tolist()
group_jid = group_jid[0]

group_name = dfs_groupJid['025587043275878954@g.us']['group_name'].unique().tolist()
group = group_name[0]

package_name = dfs_groupJid['025587043275878954@g.us']['package_name'].unique().tolist()
package = package_name[0]

# stage_id = dfs_groupJid['025587043275878954@g.us']['stage_id'].unique().tolist()
# stage = stage_id[0]

print(group_id)
print(group_jid)
print(group)
print(package)
# print(stage)

3
025587043275878954@g.us
EduardoeNicolas-MulherdaWeb
Pacote Especial


# Gerando relatórios de um grupo

In [88]:
# # Diretório onde os arquivos CSV estão localizados
# csv_folder = r"ChatHistory/byData"

# # Listar todos os arquivos CSV na pasta
# arquivos = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

# # Criar uma função para extrair a data do nome do arquivo
# def extrair_data(arquivo):
#     # Usar uma expressão regular para encontrar a data no formato YYYY-MM-DD
#     match = re.search(r'_(\d{4}-\d{2}-\d{2})\.csv$', arquivo)
#     return match.group(1) if match else None

# # Criar uma lista de tuplas (data, arquivo) para ordenar
# dados_ordenados = [(extrair_data(arquivo), arquivo) for arquivo in arquivos]

# # Filtrar apenas arquivos válidos (com data extraída)
# dados_ordenados = [dado for dado in dados_ordenados if dado[0] is not None]

# # Ordenar pela data
# dados_ordenados.sort(key=lambda x: x[0])  # Ordena pelo primeiro item da tupla (data)

# # Exibir os arquivos ordenados
# for data, arquivo in dados_ordenados:
#     print(arquivo)


In [89]:
# df = pd.read_csv('ChatHistory/024509451509359969@g.us.csv')

# df.sample()
# df.columns

In [90]:
# group_id = df['group_id'].unique().tolist()

In [91]:
# # Obter valores 
# group_id = df['group_id'].unique().tolist()
# group_id = group_id[0]

# group_jid = df['groupJid'].unique().tolist()
# group_jid = group_jid[0]

# group_name = df['group_name'].unique().tolist()
# group = group_name[0]

# package_name = df['package_name'].unique().tolist()
# package = package_name[0]

# stage_id = df['stage_id'].unique().tolist()
# stage = stage_id[0]

# print(group_id)
# print(group_jid)
# print(group)
# print(package)
# print(stage)

In [92]:
# #obter participantes
# df_people = carregar_tabela_postgresql('people')
# df_people_groups = carregar_tabela_postgresql('people_groups')
# df_people_groups = df_people_groups[df_people_groups['group_id'] == group_id]
# df_people_groups

# df_people = pd.merge(df_people, df_people_groups, left_on='person_id', right_on='person_id', how='inner')

# # Definir as colunas que você deseja remover
# colunas_a_remover = ['person_id', 'status','created_at','updated_at','people_groups_id', 'group_id']

# # Remover as colunas do DataFrame
# df_people = df_people.drop(columns=colunas_a_remover)

# # Exibir o DataFrame unido
# df_people.sample()
# df_people

In [93]:
# df_people.dtypes

In [94]:
# df.sample()

In [95]:
# df.shape

In [96]:
# df.dtypes
# df['sender_id'] = df['sender_id'].astype(str)
# df.dtypes

## Unindo tabela projetos

In [97]:
# # df = pd.merge(df_people, df, left_on='sender_id', right_on='remote_jid', how='inner')
# df = pd.merge(df_people, df, left_on='remote_jid', right_on='sender_id', how='inner')
# df.shape

In [98]:
# df.columns

In [99]:
# # Definir as colunas que você deseja remover
# colunas_a_remover = ['project_updated_at', 'closed_at', 'grupo_created_at', 'group_id','package_id_x','package_id_y']

# # Remover as colunas do DataFrame
# df = df.drop(columns=colunas_a_remover)

In [100]:
# df.sample()


In [101]:
# project_created_at = df['project_created_at'].unique().tolist()
# project_created_at

In [102]:
# data = df['data'].unique().tolist()
# data

In [103]:
# # Carrega relatórios anteriores
# relatorios_antigo = carregar_tabela_com_where('reports', f'group_id={group_id}')
# relatorios_antigo

In [104]:
# df.to_csv('convertido.csv',index=False)

# Os agentes

In [105]:
# Carrega relatórios anteriores
relatorios_antigo = carregar_tabela_com_where('reports', f'group_id={group_id}')
relatorios_antigo

/tmp/ipykernel_1851117/3732391048.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,report_id,group_id,stage,report_name,type,report_content,created_at,updated_at


In [106]:
# Inicializar a string com as tags de abertura e fechamento
relatorios_anteriores = """<relatorios_anteriores>
"""

# Loop para percorrer cada linha do DataFrame relatorios_antigo
for index, row in relatorios_antigo.iterrows():
    # Converter a linha em uma string no formato desejado e adicionar à string 'relatorios_anteriores'
    relatorios_anteriores += f"{row.to_string(index=False)}\n"

# Fechar a tag de </relatorios_anteriores>
relatorios_anteriores += "</relatorios_anteriores>"

# Exibir o resultado
relatorios_anteriores

'<relatorios_anteriores>\n</relatorios_anteriores>'

In [107]:
dfs_groupJid['025587043275878954@g.us'].columns

Index(['message_id', 'event', 'groupJid', 'message_type', 'message_content',
       'media_url', 'reaction', 'latitude', 'longitude', 'display_name',
       'vcard', 'poll_data', 'source', 'reaction_message_id', 'quoted_message',
       'created_at', 'remote_jid', 'person_name', 'nickname', 'is_client',
       'department_name', 'group_id', 'group_name', 'project_id',
       'group_created', 'package_name'],
      dtype='object')

In [108]:
# Para obter apenas a data como um objeto datetime.date
data_inicial = dfs_groupJid['025587043275878954@g.us'].group_created.iloc[0].date()
print(data_inicial)


2024-10-04


In [109]:
# Adiciona 30 dias úteis
data_final = data_inicial + pd.offsets.BDay(30)
data_final

Timestamp('2024-11-15 00:00:00')

In [110]:
etapas = f"""<etapas>
Projeto iniciado em: {data_inicial}
Estimativa de término: {data_final}

# Etapa 01 - Prazo: 2 dias

1. Apresentação do atendimento;
2. Envio do cronograma com prazos;
3. Definição de nicho:
    3.1. indicar o uso de apenas um ou nichos que se relacionem entre si;
    2.2. caso cliente não tenha nicho em mente, indicar um.
4. Definir nome;
5. Registrar dimínio;
6. Referências de Criação de Logo;
    6.1 Identificar se cliente possui referências;
    6.2 Enviar referências;
    6.3 Definir paleta de cores;
7. Solicitar a equipe de desenvolvimento a criação de logo;
8. Solicitar ao cliente credenciais de acesso às plataformas a serem integradas:
    Exemplos: aliexpress, appmax, hostinger, facebookt, instagram, yampi, dsers, plataforma de e-comerce;
    - caso tenha domínio registrado, acesso à plataforma de registro;

# Etapa 02 - Prazo: 2 dias

1. Definir Categoria;
2. Buscar e/ou informar referências para layout do e-commerce:
    - Sugerir um de nossos templates: `https://gifthype.com.br/` e  `https://clothingoficial.com.br/`
3. Escolha da plataforma de mineração:
    Disponíveis: Dropi, Dslite, Dsers, Printful.
4.Solicitar mineração de produtos baseados no segmento e plataforma à equipe de desenvolvimento;

# Etapa 03 - Prazo: 6 dias

1. Aprovação de logo, caso solicite reajuste: Prazo de 1 dia;
2. Envio de produtos minderados - Prazo: 2 dias;
3. Envio da precificação padrão utilizada:
    - ( Valor do produto minerado + frete ) * Dólar(R$5,60) * 1,8
4. Aprovação de produtos;
5. Solicitar cadastro de produtos no e-commerce;
6. Solicitar criação de loja:
    - Requisitos:
        - Referência de layout;
        - Acessos das contas que serão integradas à loja
        - Informações do briefing comercial
        - Qualquer outra informação útil que possa agregar na criação do layout
7. Solicitar desenvolvimento  de banners:
    - Requisitos:
        - Link da loja
        - Referências para criação dos banners
        - Qualquer outra informação útil que possa agregar na criação dos banners

# Etapa 04 - Prazo: 1 dia

1. Marcar call para apresentar o ecommerce;
2. Confirmar call no Google Agenda;
3. Definir produtos para campanhas de marketing:
    - Caso não exista preferência de produtos pelo cliente, sugerir produtos para as campanhas;
    - Solicitar criação de criativos.

# Etapa 05 - Prazo: 2 dias após a call

1. Transferência da loja para o cliente:
    - Loja criada em ambiente de desenvolvimento deve ser transferida para uma conta configurada pelo cliente e ativada pelo mesmo.

# Etapa 06 - Prazo: 4 dias úteis

1. Aprovação de criativos:
    Quando aprovado, informar marketing via "Monday" que já pode-se configurar contas de anúncio
    - Requisitos:
        - Contas de anúncio configuradas

# Etapa 07 - Prazo: 4 dias úteis

1. Lançar campanhas: 
    Depois de configurar as contas, o marketing enviará ao cliente um link para que ele adicione a forma de pagamento dos anúncios.
2. Relatório de finalização do projeto

# Etapa 08 - Dia 18 ao 30 dia 

1. Acompanhamento e otimização das campanhas, 
2. Envio de feedback semanal 
</etapas>
"""

In [111]:
solicitacoes = f"""<solicitacoes>
1. OBJETIVOS - Analise o histórico de conversas no whatsapp com base em <etapas></etapas> do projeto.\n
2. GAP - Identificação de possíveis GAPs no relacionamento com o cliente.\n
3. METODOLOGIA - Resumir o que ocorreu, analisar sentimentos, identificar gaps, propor scripts e melhorias no atendimento.\n
5. RESULTADOS - Identificar cronograma e cumprimento de prazos, bem como a satisfação do cliente.\n
{etapas}</solicitacoes>\n
"""

In [112]:
controles = """
<controles>
NÍVEIS DE CONTROLE:\n
1. Entonação: Formal 
5. Foco: Você deve responder sempre com foco nos objetivos das <etapas> buscando encontrar GAPs, atingir <metas> e cumprir <prazos>
6. Línga: Escreva sempre em português do Brasil, como brasileiros especialistas em gestão de negócios constumam escrever.
7. Controle de Sentimento: Neutro. Evite superlativos como: inovador, revolucionário, etc.
8. Nível de originalidade: 10, onde 1 é pouco original e 10 é muito original. 
9. Nível de abstração: 1, onde 1 é muito concreto e 10 é muito abstrato e irreal.
10. Tempo Verbal: Escreva no presente.\n
</controles>\n
"""

In [113]:
restricoes = """<restricoes>
O QUE VOCÊ NÃO DEVE FAZER:\n
1. Criar novas informações.
2. Oferecer descontos.
</restricoes>\n
"""

In [114]:
# Modelo de relatório
template = """<template>
Relatório do Grupo: <grupo> - Etapa: <etapa>\n
Dia: <dia>\n
Prazo: <prazo>
1. Objetivos: Objetivos da etapa\n 
2. Resumo do dia: Um resumo do que houve\n
2. Análise de sentimento: Análise de sentimentos dos <clientes>\n
3. GAPs: Os gaps identificados na relação com cliente, objetivos e etapas do projeto\n
4. Pendências: As pendências do dia e da etapa\n 
5. Avaliações: Avaliações sobre o dia e etapa\n
6. Proposta de melhorias\n
</template>
"""

In [115]:
def save_to_txt(file_name, file):
    """
    Salva as saídas do agente em um arquivo .txt.

    Parameters:
    file_name (str): O nome do arquivo onde as saídas serão salvas.
    goal (str): O objetivo do agente.
    backstory (str): A história de fundo do agente.
    """
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(file)  # Grava apenas o objetivo do agente

In [116]:
agent_relator_goal = (
    "Ler CSVs com conversas de whatsapp e extrair informações específicas conforme definido nas solicitações em <solicitacoes>. "
    f"Gerar um relatório de acordo com o modelo especificado em <template>.\n {solicitacoes} {template}"
)

agent_relator_backstory = (
    "Você é um especialista nos processos de atendimento da Goodds. "
    "Sua missão é analisar as conversas entre a equipe de trabalho e os clientes e extrair informações, "
    "identificar GAPs, entender o processo e as etapas do projeto, gerar insights, propor ações e  melhorias para alcançar, "
    "além dos objetivos principais, a satisfação do cliente. "
    "Sua função é fundamental para avaliar nosso trabalho e otimizar processos. "
    "Ao responder as solicitações delimitadas por <solicitacoes></solicitacoes>, você deve levar em consideração as "
    "definições de controle em <controle></controle> e as restrições em <restricoes></restricoes>.\n\n"
    f"{solicitacoes} {restricoes} {controles}"
)

# Exibindo os resultados
# save_to_txt('agent_relator_goal',agent_relator_goal)
save_to_txt('agent_relator_backstory',agent_relator_backstory)

In [117]:
def create_agent_relator(llm, tool):
    return Agent(
        role="Relator de conversas",
        goal=agent_relator_goal,
        backstory=agent_relator_backstory,
        tools=[tool],
        verbose=True,
        memory=False,
        llm=llm
    )

In [118]:
agent_revisor_goal = (
    "Leia os dados extraídos pelo Agente Relator e verifique se o relatório foi produzido "
    "de acordo com o template proposto em <template>, com os dados solicitados em <solicitacoes>. "
    "Compare o relatório produzido pelo agente relator com relatórios anteriores se existirem "
    "em <relatorios_anteriores> Como resultado do seu trabalho, "
    "você deve retornar um relatório revisado no mesmo formato do "
    f"template proposto. {solicitacoes} {template} {relatorios_anteriores}"
)

agent_revisor_backstory = (
    "Você é um especialista em revisão de relatórios. "
    "Sua função é garantir que os dados extraídos reflitam as solicitações definidas em <solicitacoes> "
    "e estejam formatados de acordo com o template proposto em <template>. "
    "Sua atenção aos detalhes assegura que os resultados finais sejam precisos "
    f"conforme as expectativas. {solicitacoes} {template}"
)

# Exibindo os resultados
# save_to_txt('agent_revisor_goal',agent_revisor_goal)
save_to_txt('agent_revisor_backstory',agent_revisor_backstory)

In [119]:
def create_agent_revisor(llm):
    return Agent(
        role="Revisor do relatório",
        goal=agent_revisor_goal,
        backstory=agent_revisor_backstory,
        verbose=True,
        memory=False,
        llm=llm
    )

In [120]:
relator_task_description = (
    "Leia o CSV e retorne um relatório com as solicitações definidas em <solicitacoes> "
    "usando o modelo definido em <template>. "
    f"{solicitacoes} {template}"
)

relator_task_expeted_output = (
    "Relatório com as solicitações definidas em <solicitacoes>"
    "usando o modelo definido em <template>. "
    f"{solicitacoes} {template}"
)

In [121]:
def relator_task(agent_relator):
    return Task(
        description=relator_task_description,
        expected_output=relator_task_expeted_output,
        agent=agent_relator,
        output_file='agent_relator.csv'
    )

In [122]:
revisor_task_description = (
    "Revise o relatório produzido pelo agente revisor para garantir que ele esteja de acordo com o template definido em <template> " 
    " e contenha todas as informações solicitadas em <solicitacoes> "
    f"{solicitacoes} {template}"
)
revisor_task_expeted_output = (
    "Relatório revisado que esteja de acordo com o template definido em <template>"
    " e contenha todas as informações solicitações em <solicitacoes>"
    f"{solicitacoes} {template}"
)

In [123]:
def revisor_task(agent_revisor):
    return Task(
        description=relator_task_description,
        expected_output=relator_task_expeted_output,
        agent=agent_revisor,
        output_file='agent_revisor.csv'
    )

In [142]:
os.environ['OPENAI_API_KEY'] = OPEN_AI_KEY

In [143]:
gpt = ChatOpenAI(model="gpt-4")

In [145]:
# txt = os.path.join(directory_path, txt_file_name)
txt = file_path
# print(txt_file_name)
# print(txt_folder)
print(txt)
# txt_tool = CSVSearchTool(txt)
txt_tool = TXTSearchTool(txt)

2024-10-06 11:53:53,360 - 140663592171328 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
2024-10-06 11:53:53,374 - 140663592171328 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
2024-10-06 11:53:53,381 - 140663592171328 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
2024-10-06 11:53:53,395 - 140663592171328 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event CollectionGetEvent: module 'chromadb' has no attribute 'get_settings'


2024-10-06 11:53:53,533 - 140663592171328 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event CollectionDeleteEvent: module 'chromadb' has no attribute 'get_settings'


conversas/output.txt


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.09it/s]


In [146]:
# relator
agent_relator = create_agent_relator(gpt,txt_tool)
task_relator = relator_task(agent_relator)
# revisor
agent_revisor = create_agent_revisor(gpt)
task_revisor = revisor_task(agent_revisor)

crew = Crew(
    agents=[agent_relator,agent_revisor],
    tasks=[task_relator,task_revisor],
    process=Process.sequential
)

ipt = {
    "solicitacoes": solicitacoes,
    "template": template,
    "restricoes": restricoes,
    "controles": controles,
}

results = crew.kickoff(inputs=ipt)

# Agent: CSV Reader
## Task: Leia o CSV e retorne um relatório com as solicitações definidas em <solicitacoes> usando o modelo definido em <template>. <solicitacoes>
1. OBJETIVOS - Analise o histórico de conversas no whatsapp com base em <etapas></etapas> do projeto.

2. GAP - Identificação de possíveis GAPs no relacionamento com o cliente.

3. METODOLOGIA - Resumir o que ocorreu, analisar sentimentos, identificar gaps, propor scripts e melhorias no atendimento.

5. RESULTADOS - Identificar cronograma e cumprimento de prazos, bem como a satisfação do cliente.

<etapas>
Projeto iniciado em: 2024-10-04
Estimativa de término: 2024-11-15 00:00:00

# Etapa 01 - Prazo: 2 dias

1. Apresentação do atendimento;
2. Envio do cronograma com prazos;
3. Definição de nicho:
    3.1. indicar o uso de apenas um ou nichos que se relacionem entre si;
    2.2. caso cliente não tenha nicho em mente, indicar um.
4. Definir nome;
5. Registrar dimínio;
6. Referências de Criação de Logo;
    6.1 Identificar se

2024-10-06 11:54:35,295 - 140663592171328 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event CollectionQueryEvent: module 'chromadb' has no attribute 'get_settings'




# Agent: CSV Reader
## Thought: Primeiro, preciso analisar o conteúdo do CSV e identificar as informações referentes às etapas do projeto. Como vou buscar essas informações no CSV, usarei a ferramenta "Search a txt's content" para identificar as informações relevantes.
## Using tool: Search a txt's content
## Tool Input: 
"{\"search_query\": \"etapas do projeto\"}"
## Tool Output: 
Relevant Content:
Pacote Global Dois Em Um, stage_id: 1, project_name: Maiara-Mayavant, store_name: , domain: , url: , niche: , logo: , mined_urls: , contact_email: , social_media_links: , status: ativo, description_y: , project_created_at: 2024-02-19 02:23:17.387, estimated_deadline:

Dois Em Um, stage_id: 1, project_name: Maiara-Mayavant, store_name: , domain: , url: , niche: , logo: , mined_urls: , contact_email: , social_media_links: , status: ativo, description_y: , project_created_at: 2024-02-19 02:23:17.387, estimated_deadline:

stage_id: 1, project_name: Maiara-Mayavant, store_name: , domain: , url

In [75]:
csv_folder = './'

In [76]:
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

In [ ]:
csv_files

In [ ]:
csv

In [79]:
# lista ára armazenar resultados
all_ralatorios = []

In [ ]:
for csv_file_name in csv_files:
    # gpt = ChatOpenAI(model="gpt-4") # a mente do agente
    # Definir a chave da API como variável de ambiente diretamente no script
    os.environ['OPENAI_API_KEY'] = "sk-dGwU-tTvhnrFog5R78lzLlP5FjaOOlWsz_Qbb4CdgoT3BlbkFJ_iRd_S5W91m7pOY9i8oT_E8wHZwgK2EiqMROjF0iIA"

    # Agora inicialize o gpt e as ferramentas normalmente
    gpt = ChatOpenAI(model="gpt-4")
    csv = os.path.join(csv_folder, csv_file_name)
    print(csv_file_name)
    print(csv_folder)
    print(csv)
    csv_tool = CSVSearchTool(csv)
    
    
    # relator
    agent_relator = create_agent_relator(gpt,csv_tool)
    task_relator = relator_task(agent_relator)
    # revisor
    agent_revisor = create_agent_revisor(gpt)
    task_revisor = revisor_task(agent_revisor)
    
    crew = Crew(
        agents=[agent_relator,agent_revisor],
        tasks=[task_relator,task_revisor],
        process=Process.sequential
    )
    
    ipt = {
        "solicitacoes": solicitacoes,
        "template": template,
        "restricoes": restricoes,
        "controles": controles,
    }
    
    results = crew.kickoff(inputs=ipt)
    
    # # Conectar ao banco de dados
    # conn = psycopg2.connect(
    #     dbname=banco_de_dados,
    #     user=usuario,
    #     password=senha,
    #     host=host,
    #     port=porta
    # )

    # # Criar um cursor para executar comandos
    # cursor = conn.cursor()
    
    # # Definir a query de inserção
    # insert_query = """
    # INSERT INTO reports (group_id, report_content)
    # VALUES (%s, %s)
    # """

    # try:
    #     # Executar a query de inserção
    #     cursor.execute(insert_query, (group_id, results))
    #     print(f"Linhas afetadas: {cursor.rowcount}")  # Verifica quantas linhas foram afetadas

    # except Exception as e:
    #     print(f"Erro ao inserir grupo: {e}")

    # # Commit para salvar as inserções
    # conn.commit()

    # # Fechar o cursor e a conexão
    # cursor.close()
    # conn.close()
    
    all_ralatorios.append(results)

final_output = {"relatorios": all_ralatorios}

now = datetime.now
formatted_date = now.strftime("%d_%m_%Y")

file_name = f"relatorios_{formatted_date}.txt"

with open(file_name, 'w') as file:
    (final_output, file,)

print(f"Arquivo '{file_name}' foi criado com sucesso!")



In [ ]:
final_output

In [ ]:
# Chame a função now() corretamente
now = datetime.now()  # Adicionar parênteses para chamar a função

# Agora você pode usar o método strftime no objeto datetime retornado por now()
formatted_date = now.strftime("%d_%m_%Y")

file_name = f"relatorios_{formatted_date}.txt"

with open(file_name, 'w') as file:
    file.write(str(final_output))  # Corrigir a escrita para salvar o dicionário como string

print(f"Arquivo '{file_name}' foi criado com sucesso!")


In [ ]:
# Saida 2
file_name2 = 'output.txt'

with open(file_name2, 'w') as file:
    for key, value in final_output.items():
        file.write(f'{key}: {value}\n')

print(f"Dados salvos em {file_name2}")